# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [1]:
# Run this cell first!

from helpers import Map, load_map_10, load_map_40, show_map
import math

%load_ext autoreload
%autoreload 2

### Map Basics

In [2]:
map_10 = load_map_10()
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [4]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [5]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [1, 4, 3],
 [1, 2, 5, 4],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [6]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)


### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [7]:
print(map_40.roads[5])

[32, 16, 14]


In [8]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

## The Algorithm
### Writing your algorithm
The algorithm you write will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`

```bash
> shortest_path(map_40, 5, 34)
[5, 16, 37, 12, 34]
```

In [9]:
# Do not change this cell
# When you write your methods correctly this cell will execute
# without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M 
        self.start= start
        self.goal = goal 
        self.closedSet = self.create_closedSet() if goal != None and start != None else None 
        self.openSet = self.create_openSet() if goal != None and start != None else None 
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None 
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
        
        
        
    def get_path(self):
        """ Reconstructs path after search """
        if self.path:
            return self.path 
        else :
            self.run_search()
            return self.path
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()
        
        while not self.is_open_empty():
            current = self.get_current_node()
            if current == self.goal: 
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)
                

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
                
        print("No Path Found")
        self.path = None
        return False

Create the following methods:

In [10]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # TODO: return a data structure suitable to hold the set of nodes already evaluated
    
    return set()

In [11]:
def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
        
        return set([self.start])
    
    raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")

In [12]:
def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another,
    # for each node. 
    
    return {}

In [13]:
def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, for each node.
    The cost of going from start to start is zero."""
    # TODO:  a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should be set to infinity.
    
    g_score = {i:math.inf for i in self.map.intersections}
    g_score[self.start] = 0
    
    return g_score
    

In [14]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO:  a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be 
    # set to infinity.
    
    fScore = {}
    
    for i in self.map.intersections:
        fScore[i] = math.inf
        
    H = self.heuristic_cost_estimate(self.start)
    fScore[self.start] = H

    return fScore

In [15]:
def _reset(self):
    """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
    self.closedSet = None
    self.openSet = None
    self.cameFrom = None
    self.gScore = None
    self.fScore = None
    self.path = self.run_search() if self.map and self.start and self.goal else None

In [16]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    self.start = None
    self.goal = None
    # TODO: Set map to new value. 
    
    self.map = M



In [17]:
def set_start(self, start):
    """Method used to set start attribute """
    self._reset(self)
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    
    self.start = start


In [18]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value. 
    
    self.goal = goal

In [19]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the open set with the lowest value of f(node).
    
    fScore = math.inf
    current_node = 0
    for node in self.openSet:
        if self.fScore[node] < fScore:
            current_node = node
            fScore = self.fScore[node]

    return current_node


In [20]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    
    return self.map.roads[node]


In [21]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    # TODO: Return the g Score of a node
    
    return self.gScore[node] 


In [22]:
def get_tenative_gScore(self, current, neighbor):
    """Returns the tenative g Score of a node"""
    # TODO: Return the tenative g Score of the current node 
    # plus distance from the current node to it's neighbors
    
    
    return self.distance(current, neighbor) + self.gScore[current]

In [23]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    
    return len(self.openSet) == 0


In [24]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distance
    
    x1, y1 = self.map.intersections[node_1]
    x2, y2 = self.map.intersections[node_2]
    a2 = math.pow((x1 - x2), 2)
    b2 = math.pow((y1 - y2), 2)
    c2 = a2 + b2
    
    return math.sqrt(c2)

In [25]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    
    return self.distance(node, self.goal)


In [26]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    
    G = self.gScore[node]
    H = self.heuristic_cost_estimate(node)
    F = G + H
    
    return F
    

In [27]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    
    self.cameFrom[neighbor] = current
    self.gScore[neighbor] = self.get_tenative_gScore(current, neighbor)
    self.fScore[neighbor] = self.calculate_fscore(neighbor)
    


In [28]:
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner._reset = _reset
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.get_tenative_gScore = get_tenative_gScore
PathPlanner.is_open_empty = is_open_empty
PathPlanner.distance = distance
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

In [29]:
planner = PathPlanner(map_40, 5, 34)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

current: 5
neighbor: 32
tent: 0.24522732199439867
actual: inf
{0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: 0, 6: inf, 7: inf, 8: inf, 9: inf, 10: inf, 11: inf, 12: inf, 13: inf, 14: inf, 15: inf, 16: inf, 17: inf, 18: inf, 19: inf, 20: inf, 21: inf, 22: inf, 23: inf, 24: inf, 25: inf, 26: inf, 27: inf, 28: inf, 29: inf, 30: inf, 31: inf, 32: 0.24522732199439867, 33: inf, 34: inf, 35: inf, 36: inf, 37: inf, 38: inf, 39: inf} 

current: 5
neighbor: 16
tent: 0.05024121466351104
actual: inf
{0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: 0, 6: inf, 7: inf, 8: inf, 9: inf, 10: inf, 11: inf, 12: inf, 13: inf, 14: inf, 15: inf, 16: 0.05024121466351104, 17: inf, 18: inf, 19: inf, 20: inf, 21: inf, 22: inf, 23: inf, 24: inf, 25: inf, 26: inf, 27: inf, 28: inf, 29: inf, 30: inf, 31: inf, 32: 0.24522732199439867, 33: inf, 34: inf, 35: inf, 36: inf, 37: inf, 38: inf, 39: inf} 

current: 5
neighbor: 14
tent: 0.14101456789585137
actual: inf
{0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: 0, 6: inf, 7: inf, 8

### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [30]:
from test import test

test(PathPlanner)

current: 5
neighbor: 32
tent: 0.24522732199439867
actual: inf
{0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: 0, 6: inf, 7: inf, 8: inf, 9: inf, 10: inf, 11: inf, 12: inf, 13: inf, 14: inf, 15: inf, 16: inf, 17: inf, 18: inf, 19: inf, 20: inf, 21: inf, 22: inf, 23: inf, 24: inf, 25: inf, 26: inf, 27: inf, 28: inf, 29: inf, 30: inf, 31: inf, 32: 0.24522732199439867, 33: inf, 34: inf, 35: inf, 36: inf, 37: inf, 38: inf, 39: inf} 

current: 5
neighbor: 16
tent: 0.05024121466351104
actual: inf
{0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: 0, 6: inf, 7: inf, 8: inf, 9: inf, 10: inf, 11: inf, 12: inf, 13: inf, 14: inf, 15: inf, 16: 0.05024121466351104, 17: inf, 18: inf, 19: inf, 20: inf, 21: inf, 22: inf, 23: inf, 24: inf, 25: inf, 26: inf, 27: inf, 28: inf, 29: inf, 30: inf, 31: inf, 32: 0.24522732199439867, 33: inf, 34: inf, 35: inf, 36: inf, 37: inf, 38: inf, 39: inf} 

current: 5
neighbor: 14
tent: 0.14101456789585137
actual: inf
{0: inf, 1: inf, 2: inf, 3: inf, 4: inf, 5: 0, 6: inf, 7: inf, 8

## Questions

**Instructions**  Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask questions. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

- How would you explain A-Star to a family member(layman)?

** ANSWER **: A-star is a search algorithm. It is used in various contexts, one of which is finding the shortest path between two places. This is what Google Maps uses to give users the shortest path to their destination. 

When searching for the shortest path, the algorithm is constantly doing roughly 2 things. It decides which place to check next and checks to see if that place is the destination. If it is the destination, it will stop because it has found the destination. If it is not, it will repeat those 2 steps.

From any location or starting point, there are a number of places it can check next. For instance, if it starts on the corner of 38th and 8th in New York City and it is searching for the shortest path to 40th and 8th, there are 4 places it can check next: 38th and 9th, 38th and 7th, 39th and 8th, and finally 37th and 8th. It can't immediately check 40th and 8th, because these places are not directly connected. It must first check 39th and 8th, then it can check 40th and 8th.

The real trick of A-star is in how it decides which place to check next. This decision making process has to do 2 things: it has to make sure it is following the shortest path and it has to narrow the search space. Narrowing the search space means the algorithm is not going to check every single place when searching; it will eliminate some places as irrelevant to check. This is important because when searching for the shortest route from New York to LA there are myriad places to check. Without narrowing the search space the algorithm could take a long time to find the route.

A-star takes care of both of these problems by taking 2 factors into account when deciding which place to check next: how far the next place is from the current place and how far the next place is from the goal. The first factor, how far the next place is from the current place, ensures that the path will be the shortest. This is because the algorithm is always checking the shortest path before checking if the next place on that path is the goal. When it eventually finds the goal, it will have done so after checking the shortest path. The second factor, how far the next place is from the goal, helps guide the search towards the goal. This helps narrow the search space by preventing the algorithm from looking in places that are close to the current place (satisfying the first factor), but moving away from the goal.

When looking for the shortest path or closest next place, it doesn't only have to look at distance. It could consider a high traffic road to be long or a road with an accident to be long, while it can consider a longer road with no traffic or traffic lights to be short.

- How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

** ANSWER **: Uniform cost search only takes into g(n) when making it's decision. This differs by not using a heuristic value. It can also be thought of as using a heuristic value of 0. Both of these algorithms are guaranteed to find the shortest path, given A* is using an admissible heuristic.

Best first search using only the heuristic value to make it's decision. This means the algorithm focuses soley on the nodes that are closes to the goal. This can run into problems when dealing with obsticles. 

- What is a heuristic?

** ANSWER **: A heuristic is a value that guides the algorithm to making the best decision. In the case of A*, the heuristic comes in the form of an estimated distance to the goal from the available nodes. In essence this allows the algorithm to take into account not only how far the available nodes are from the current node, but whether or not the node gets closer to or further from the goal. This effectively narrows the search space allowing the optimal path to be found quicker than if the heuristic had not been taken into account.

- What is a consistent heuristic?

** ANSWER **: A consistent heuristic is one that satisfies the inequality |h(n') - h(n)| <= d(n', n). This is essentially saying that the change in heuristic value for any movement between nodes never changes more than the actual distance moved between the nodes. For instance, if h(n) = 7 and h(n') = 0 then the difference in the estimated distance to the goal changes by 7 between those 2 nodes. If the actual distance between those nodes is 1, then it makes no sense that it would move 1 measure closer to the goal, but estimate that it moved 7 measures closer.

Using a consistent heuristic ensures that when a node is expanded for the first time, the path taken to that node was the shortest possible path. The inconsistent heuristic can overestimate the distance between 2 nodes, thereby potentially eliminating the shortest path to that node.

- What is a admissible heuristic? 

** ANSWER **: An admissible heuristic is one that satisfies the inequality h(n) <= g(n, goal). In other words, the value of the heuristic never overestimates the actual shortest path cost from the node to the goal. This is important because when a heuristic is inadmissible, or h(n) > g(n, goal), it does not guarantee that the algorithm will find the shortest path. This happens because the algorithm could overestimate the optimal path causing it to prefer a longer path instead. As the heuristic value approaches 0 the algorithm gets closer and closer to the uniform cost algorithm as only g(n) is taken into account when h(n) = 0. The optimal, admissible heuristic is one where h(n) == g(n, goal). This effectively would give the algorithm perfect information to work with, allowing it to find the shortest path in the shortest amount of time, or smallest search area.

- ___ admissible heuristic are consistent.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: Some

- ___ Consistent heuristic are admissible.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: All